先算出漢字的頻率分布。
我們先定義一個function，input是個list，output是個dictionary：


In [1]:
def list2freqdict(mylist):
  mydict = dict()
  for ch in mylist:
    mydict[ch] = mydict.get(ch, 0) +1
  return mydict

sentence='吃葡萄不吐葡萄皮，不吃葡萄倒吐葡萄皮。'
chlist=[ch for ch in sentence]

print(chlist)
chfreqdict=list2freqdict(chlist)
print(chfreqdict)

['吃', '葡', '萄', '不', '吐', '葡', '萄', '皮', '，', '不', '吃', '葡', '萄', '倒', '吐', '葡', '萄', '皮', '。']
{'吃': 2, '葡': 4, '萄': 4, '不': 2, '吐': 2, '皮': 2, '，': 1, '倒': 1, '。': 1}


原因是dictionary並沒有順序的概念，因此印出來的順序是隨機。而且有時候我們只對高頻字有興趣，或是漢字太多，這時應該要有排序，然後只印出次數大於m以上的漢字的次數，或根據次數，印出前n個漢字的次數。

In [2]:
from operator import itemgetter
chfreqsorted=sorted(chfreqdict.items(), key=itemgetter(1), reverse=True)
print(chfreqsorted)

[('葡', 4), ('萄', 4), ('吃', 2), ('不', 2), ('吐', 2), ('皮', 2), ('，', 1), ('倒', 1), ('。', 1)]


我們想限定只印出前幾個最高次數的漢字及其次數，可以用list indexation。若想只印出出現次數高於某個數字的漢字及其次數，則必須另起爐灶，這邊我們會用到一個list的append，資料形態是list of lists，而append一次只能附加一個元素，因此要寫成append((ch,num))而不是append(ch,num)：前者是說：附加一個叫做(ch,num)的元素，後者是說：附加兩個元素叫做ch與num。這邊我們只印出前5個結果，以及出現次數大於1的結果：

In [3]:
chfreqsorted2 = chfreqsorted[:5]

chfreqsorted3 = list()

for ch, num in chfreqsorted:
  if num >1 :
    chfreqsorted3.append((ch, num))
print("出現頻率前五名：",chfreqsorted2)
print("出現頻率大於一的字：",chfreqsorted3)

出現頻率前五名： [('葡', 4), ('萄', 4), ('吃', 2), ('不', 2), ('吐', 2)]
出現頻率大於一的字： [('葡', 4), ('萄', 4), ('吃', 2), ('不', 2), ('吐', 2), ('皮', 2)]


In [4]:
def list2bigram(mylist):
    return [mylist[i:i+2] for i in range(0,len(mylist)-1)]

def list2trigram(mylist):
    return [mylist[i:i+3] for i in range(0,len(mylist)-2)]

chbigram=list2bigram(chlist)
chtrigram=list2trigram(chlist)
print(chbigram)
print(chtrigram)

[['吃', '葡'], ['葡', '萄'], ['萄', '不'], ['不', '吐'], ['吐', '葡'], ['葡', '萄'], ['萄', '皮'], ['皮', '，'], ['，', '不'], ['不', '吃'], ['吃', '葡'], ['葡', '萄'], ['萄', '倒'], ['倒', '吐'], ['吐', '葡'], ['葡', '萄'], ['萄', '皮'], ['皮', '。']]
[['吃', '葡', '萄'], ['葡', '萄', '不'], ['萄', '不', '吐'], ['不', '吐', '葡'], ['吐', '葡', '萄'], ['葡', '萄', '皮'], ['萄', '皮', '，'], ['皮', '，', '不'], ['，', '不', '吃'], ['不', '吃', '葡'], ['吃', '葡', '萄'], ['葡', '萄', '倒'], ['萄', '倒', '吐'], ['倒', '吐', '葡'], ['吐', '葡', '萄'], ['葡', '萄', '皮'], ['萄', '皮', '。']]


In [6]:
from collections import Counter
A = ['我們會用到 Collections 中的 namedtuple 與 Counter。namedtuple 提供的功能是像 C 語言的 struct，能夠清楚表達一個資料的結構；而 Counter 則是能夠快速地幫助我們計算數量，例如 list 中每個元素的數量各有幾個']
for i in A:
  print(Counter(i))

Counter({' ': 13, 't': 8, 'e': 7, 'l': 5, 'n': 5, '的': 5, 'u': 5, 'C': 4, 'o': 4, 's': 3, 'r': 3, '能': 3, '個': 3, '我': 2, '們': 2, 'c': 2, 'i': 2, '中': 2, 'a': 2, 'm': 2, 'd': 2, 'p': 2, '是': 2, '，': 2, '夠': 2, '數': 2, '量': 2, '會': 1, '用': 1, '到': 1, '與': 1, '。': 1, '提': 1, '供': 1, '功': 1, '像': 1, '語': 1, '言': 1, '清': 1, '楚': 1, '表': 1, '達': 1, '一': 1, '資': 1, '料': 1, '結': 1, '構': 1, '；': 1, '而': 1, '則': 1, '快': 1, '速': 1, '地': 1, '幫': 1, '助': 1, '計': 1, '算': 1, '例': 1, '如': 1, '每': 1, '元': 1, '素': 1, '各': 1, '有': 1, '幾': 1})


#建立完整的N-gram架構

In [7]:
from collections import Counter, namedtuple
import json
import re
import os

**引入資料集**

使用的資料集是政府資料開放平台提供的 🔗桃園市官網市政新聞，由於 N-gram 所產生的結果會根據資料集而有所不同，因此在選擇資料集時必須評估該資料集是否合適。假設使用的資料集中包含許多火星文，根據 N-gram 產生的語言模型，其結果將會與火星文相關。

In [10]:
DATASET_DIR = '/content/WebNews.json'
with open(DATASET_DIR, encoding ='utf8') as f:
  dataset = json.load(f)

In [11]:
seg_list = list(map(lambda d: d['detailcontent'], dataset))#存入list中
rule = re.compile(r"[^\u4e00-\u9fa5]") 
#\u4e00-\u9fa5是用来判斷是不是中文的一個條件，採用的是unicode編碼,^\u4e00-\u9fa5表示除了中文之外
seg_list = [rule.sub('', seg) for seg in seg_list]#從之前存入的list中一條條讀取，中文以外的的刪除

In [12]:
def ngram(documents , N= 2):
  ngram_prediction = dict()
  total_grams = list() #蒐集分子
  words = list() #蒐集分母
  Word = namedtuple('Word', ['word', 'prob'])
  for doc in documents:
    split_words = ['<s>'] + list(doc) + ['</s>']
    #計算分子
    [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
    #計算分母
    [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]
  
  total_word_counter = Counter(total_grams) 
  word_counter = Counter(words)

  for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
            
        next_word_prob = total_word_counter[key]/word_counter[key[:N-1]]
        w = Word(key[-1], '{:.3g}'.format(next_word_prob))
        ngram_prediction[word].add(w)
        
  return ngram_prediction

# 訓練模型與排序
使用的模型是 trigram，也就是計算接在兩個字之後第三個字的機率。接著，可以對結果進行排序，因此在預測下一個字時，能夠直接取得前幾個最高機率的字。

In [13]:
tri_prediction = ngram(seg_list, N=3)
for word, ng in tri_prediction.items():
    tri_prediction[word] = sorted(ng, key=lambda x: x.prob, reverse=True)

In [14]:
text = '桃園'
next_words = list(tri_prediction[text])[:5] #出現機率前五名
for next_word in next_words:
    print(f'next word: {next_word.word}, probability: {next_word.prob}')

next word: 市, probability: 0.341
next word: 區, probability: 0.042
next word: 的, probability: 0.0338
next word: 地, probability: 0.0165
next word: 航, probability: 0.0134
